# Azure AI Agent सेवा र सेम्यान्टिक कर्नेल प्रयोग गरेर RAG

यो कोड स्निपेटले `Azure AI Agent Service` र `Semantic Kernel` प्रयोग गरेर पुनःप्राप्ति-संवर्धित उत्पादन (RAG) को लागि Azure AI एजेन्ट सिर्जना र व्यवस्थापन गर्ने तरिका देखाउँछ। एजेन्टले पुनःप्राप्त गरिएको सन्दर्भको आधारमा प्रयोगकर्ताको प्रश्नहरू प्रक्रिया गर्छ र त्यस अनुसार सही उत्तर प्रदान गर्छ।


SQLite संस्करण समस्या समाधान  
यदि तपाईंले यो त्रुटि सामना गर्नुभयो:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

आफ्नो नोटबुकको सुरुमा यो कोड ब्लक अनकमेण्ट गर्नुहोस्:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### प्याकेजहरू आयात गर्दै  
तलको कोडले आवश्यक प्याकेजहरू आयात गर्दछ:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# सेम्यान्टिक कर्नेल र Azure AI एजेन्ट सेवा प्रयोग गरेर रिट्रिभल-अग्मेन्टेड जेनेरेसन

यो नमुनाले **Azure AI एजेन्ट सेवा** प्रयोग गरेर **रिट्रिभल-अग्मेन्टेड जेनेरेसन (RAG)** कसरी गर्ने भनेर देखाउँछ, जसमा भाषा मोडेललाई अपलोड गरिएको कागजातबाट प्राप्त डोमेन-विशिष्ट सन्दर्भसँग संयोजन गरिन्छ।

### यो कसरी काम गर्छ

1. **कागजात अपलोड**: एक markdown फाइल (document.md) जसमा जानकारी (Contoso को यात्रा बीमा नीति) समावेश छ, एजेन्ट सेवामा अपलोड गरिन्छ।

2. **भेक्टर स्टोर सिर्जना**: कागजातलाई भेक्टर स्टोरमा अनुक्रमणिका गरिन्छ ताकि यसको सामग्रीमा सेम्यान्टिक खोज सम्भव होस्।

3. **एजेन्ट कन्फिगरेसन**: `gpt-4o` मोडेल प्रयोग गरेर एजेन्टलाई निम्न कडा निर्देशनसहित सुरु गरिन्छ:
   - केवल कागजातबाट प्राप्त सामग्रीमा आधारित प्रश्नहरूको उत्तर दिनुहोस्।
   - यदि प्रश्न दायराबाहिर छ भने उत्तर दिन अस्वीकार गर्नुहोस्।

4. **फाइल सर्च टूल एकीकरण**: `FileSearchTool` लाई एजेन्टसँग दर्ता गरिन्छ, जसले मोडेललाई अनुक्रमणिका गरिएको कागजातबाट प्रासंगिक अंशहरू खोज्न र पुनःप्राप्त गर्न सक्षम बनाउँछ।

5. **प्रयोगकर्ता अन्तरक्रिया**: प्रयोगकर्ताहरूले प्रश्न सोध्न सक्छन्। यदि कागजातमा प्रासंगिक जानकारी भेटिन्छ भने, एजेन्टले आधारभूत उत्तर उत्पन्न गर्छ।  
   यदि जानकारी पर्याप्त छैन भने, एजेन्टले स्पष्ट रूपमा कागजातमा पर्याप्त जानकारी नभएको जवाफ दिन्छ।


### मुख्य कार्य

```python
def main():
    # यो मुख्य कार्य हो जसले कार्यक्रम सुरु गर्छ
    print("कार्यक्रम सुरु हुँदैछ...")
    result = perform_calculation(5, 10)
    print(f"परिणाम: {result}")
```

### Helper Function

```python
def perform_calculation(a, b):
    # दुई संख्याको योगफल गणना गर्ने सहायक कार्य
    return a + b
```

### Instructions

1. माथिको कोडलाई आफ्नो परियोजनामा समावेश गर्नुहोस्।
2. `main()` कार्यलाई चलाउनुहोस्।
3. सुनिश्चित गर्नुहोस् कि `perform_calculation` ले सही परिणाम फर्काउँछ। 

[!NOTE] यो कोड केवल उदाहरणको लागि हो। उत्पादन वातावरणमा प्रयोग गर्नु अघि परीक्षण गर्नुहोस्।


Azure CLI प्रयोग गरेर `az login` पहिले चलाउनुहोस् ताकि `DefaultAzureCredential` प्रयोग गर्दा उचित प्रमाणीकरण सन्दर्भ प्रदान गरियोस्। Azure AI Agent Service ले API कुञ्जीहरू प्रयोग गर्दैन।


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

ठीक छ, कृपया मलाई अनुवाद गर्नका लागि Markdown फाइल प्रदान गर्नुहोस्।



---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी यथासम्भव सटीकता सुनिश्चित गर्न प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादहरूमा त्रुटि वा अशुद्धता हुन सक्छ। यसको मूल भाषामा रहेको मूल दस्तावेज़लाई आधिकारिक स्रोत मानिनुपर्छ। महत्त्वपूर्ण जानकारीका लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याको लागि हामी जिम्मेवार हुने छैनौं।
